# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Forsooth, to reach the other side!


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! It's nice to meet you. How are you doing today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have the ability to remember personal information about users. How can I assist you today?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa! If you ever need a reminder again, just let me know.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [9]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [10]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

In [11]:
#exam 1
# Bài tập: sử dụng các chiến lược prompt đã học để giải quyết bài toán sau
prompt='''
 Giờ Văn cô giáo trả bài kiểm tra.
 Bốn bạn Tuấn, Hùng, Lan, Quân ngồi cùng bàn đều đạt điểm 8 trở lên.
 Giờ ra chơi Phương hỏi điểm của 4 bạn, Tuấn trả lời :
  - Lan không đạt điểm 10, mình và Quân không đạt điểm 9 còn Hùng không đạt điểm 8.
 Hùng thì nói :
  - Mình không đạt điểm 10, Lan không đạt điểm 9 còn Tuấn và Quân đều không đạt điểm 8.

Bạn hãy cho biết mỗi người đã đạt mấy điểm?.
Let's think step by step
Step1: Có các điểm 8,9 và 10 nên điểm của các bạn chỉ có thể là 8 hoặc 9 hoặc 10.
Step2: Nếu từng bạn trả lời là không đạt điểm X (X có thể là điểm 8,9,10) hãy liệt kê đủ điểm X của từng bạn không đạt được theo lời của từng bạn đã nói.
Step3: Từ đó ta sẽ loại trừ các phương án có X mà ta đã liệt kê trong lời nhắc ta đã tổng hợp vì đó là phương án sai. Ngoài các điểm X thì điểm còn lại trong 3 số 8,9,10 và khác X chính là điểm cần tìm.
'''

response = get_completion(prompt)
print(response)

Theo lời của Tuấn:
- Lan không đạt điểm 10, nghĩa là Lan đạt điểm 8 hoặc 9.
- Tuấn và Quân không đạt điểm 9, nghĩa là Tuấn và Quân đạt điểm 8 hoặc 10.
- Hùng không đạt điểm 8, nghĩa là Hùng đạt điểm 9 hoặc 10.

Theo lời của Hùng:
- Hùng không đạt điểm 10, nghĩa là Hùng đạt điểm 8 hoặc 9.
- Lan không đạt điểm 9, nghĩa là Lan đạt điểm 8 hoặc 10.
- Tuấn và Quân đều không đạt điểm 8, nghĩa là Tuấn và Quân đều đạt điểm 9 hoặc 10.

Từ đó, ta có thể suy ra điểm của mỗi người như sau:
- Lan: 9 điểm
- Tuấn: 10 điểm
- Hùng: 8 điểm
- Quân: 9 điểm

Vậy là đã tìm ra được điểm của mỗi người.


In [12]:
#exam2
# Bài tập: thực hiện các chiến lược generate knowledge prompt để giải quyết câu hỏi sau:
knowledge1='''
Chiều cao trung bình của cầu thủ bóng đá chuyên nghiệp thường thấp hơn so với cầu thủ bóng rổ, trung bình dao động từ 1,75m đến 1,85m, tùy thuộc vào giải đấu và quốc gia. Cầu thủ bóng đá thường không cần chiều cao quá cao để thi đấu hiệu quả, thay vào đó sự nhanh nhẹn, linh hoạt và thể lực được ưu tiên.
'''
knowledge2 = '''
Chiều cao trung bình của cầu thủ bóng rổ chuyên nghiệp thường dao động từ 1,98m đến 2,13m, tùy thuộc vào giải đấu và vị trí thi đấu. Ví dụ, các cầu thủ chơi ở NBA (National Basketball Association) thường có chiều cao lớn hơn so với các cầu thủ thi đấu ở các giải đấu nhỏ hơn.
'''

prompt=f'''
Question: chiều cao trung bình của cầu thủ bóng đá cao hơn một cầu thủ bóng rổ. Đúng hay sai ?
Knowledge: {knowledge1}
Knowledge: {knowledge2}
Explain and Answer:
'''

response = get_completion(prompt)
print(response)

Sai. Chiều cao trung bình của cầu thủ bóng rổ cao hơn so với cầu thủ bóng đá. Cầu thủ bóng rổ chuyên nghiệp thường có chiều cao dao động từ 1,98m đến 2,13m, trong khi chiều cao trung bình của cầu thủ bóng đá chuyên nghiệp thường thấp hơn, từ 1,75m đến 1,85m.
